In [4]:
from PIL import Image 
import requests 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
from transformers import AutoProcessor 
import base64

model_id = "microsoft/Phi-3-vision-128k-instruct" 

# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2') # use _attn_implementation='eager' to disable flash attention
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype="auto", _attn_implementation='eager') # use _attn_implementation='eager' to disable flash attention

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 



2024-07-17 03:42:04.044469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 03:42:04.044552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 03:42:04.934340: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 03:42:06.272606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 03:42:11.423590: W tensorflow/compiler/tf2

RuntimeError: Failed to import transformers.models.gemma2.configuration_gemma2 because of the following error (look up to see its traceback):
No module named 'transformers.models.gemma2.configuration_gemma2'

In [15]:
# messages = [ 
#     {"role": "user", "content": "<|image_1|>\nIdentify all the cars in the image and their pixel location, including visible vehicles in the background."}]


messages = [ 
    {"role": "user", "content": "<|image_1|>\nCount all the cars in the image and describe them. If possible include their pixel locations."}]


In [ ]:
image_path = "./LMMs_object_tracking/dashcam/rgb-images/sample_images/image1.jpg" 

image = Image.open(r"../LLMs_object_tracking/dashcam/rgb-images/sample_images/image1.jpg")
image


In [16]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 
# inputs = processor(prompt, [image], return_tensors="pt")

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)

/home/ladozal/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


There are at least 10 cars visible in the image. They are of various colors including yellow, white, and black. The pixel locations for each car cannot be accurately determined due to the image's resolution and the angle of the camera.


## Evaluation of single image
phi3 cannot identify anything when using the prompt, *Identify all the cars in the image and their pixel location, including visible vehicles in the background.*

It will find cars with the prompt: *Identify all the cars in the image.*:
- There are multiple cars in the image, including a yellow taxi in the foreground and various other vehicles in the background.

It will count cars pretty well with the prompt: *Count all the cars in the image.*:
- There are at least 10 cars visible in the image.

It will count the cars and describe them separately. Not good at pixel location using *ount all the cars in the image and describe them. If possible include their pixel locations.* :
- There are at least 10 cars visible in the image. They are of various colors including yellow, white, and black. The pixel locations for each car cannot be accurately determined due to the image's resolution and the angle of the camera.

